# Dumpulator VEH
> A deeper look at the Dumpulator advanced emulation capabilities

- toc: true 
- badges: true
- categories: [guloader,emulation,dumpulator,veh,exceptions]


## Overview

We are going to take a closer look at how to emulate exceptions with [Dumpulator](https://github.com/mrexodia/dumpulator) using a recent Guloader sample as an example.


Guloader SHA256 = `E3A8356689B97653261EA6B75CA911BC65F523025F15649E87B1AEF0071AE107` [malshare](https://malshare.com/sample.php?action=detail&hash=e3a8356689b97653261ea6b75ca911bc65f523025f15649e87b1aef0071ae107)

### Guloader VEH 
The sample we are analyzing uses a VEH trick to redirect program flow... basically they set their own VEH then use breakpoints `0xcc` in the code to trigger a small algorithm in the VEH. This aglorithm is used to caculate a new EIP based on the byte following the breakpoint.

```
EIP = EIP + ([EIP + 1] ^ 0x8f)
```

### Obfuscated Strings
In addition to the VEH program flow obfuscation the sample also uses a two-stage string encryption algorithm. First, a function is used to fill a buffer with the encrypted string data using some unfolded constants and a bit of arithmatic. Then the buffer is passed to a string decryption algorithm. The first buffer fill function is often obfuscated using the VEH program flow redirection.

## Dumpulator String Decryption


In [1]:
from dumpulator import Dumpulator

dp = Dumpulator("/tmp/gudump.dmp")

TEB: 0x2c1000, PEB: 0x2be000
  ConsoleHandle: 0x9c
  StandardInput: 0xa0
  StandardOutput: 0xa4
  StandardError: 0xa8
Patching Wow64Transition: 770f9228 -> 76fc7000
Memory map:
      Base       Size       State                Protect                      Info
   0x10000    0x10000  MEM_COMMIT         PAGE_READWRITE
   0x20000     0x1000  MEM_COMMIT PAGE_EXECUTE_READWRITE
   0x21000    0x1f000    MEM_FREE
   0x40000    0x1d000  MEM_COMMIT          PAGE_READONLY
   0x5d000     0x3000    MEM_FREE
   0x60000    0x35000 MEM_RESERVE         PAGE_READWRITE
   0x95000     0xb000  MEM_COMMIT         PAGE_READWRITE
   0xa0000    0xfb000 MEM_RESERVE         PAGE_READWRITE
  0x19b000     0x5000  MEM_COMMIT         PAGE_READWRITE
  0x1a0000     0x4000  MEM_COMMIT          PAGE_READONLY
  0x1a4000     0xc000    MEM_FREE
  0x1b0000     0x2000  MEM_COMMIT         PAGE_READWRITE
  0x1b2000     0xe000    MEM_FREE
  0x1c0000    0x35000 MEM_RESERVE         PAGE_READWRITE
  0x1f5000     0xb000  MEM_COMMIT 

In [3]:
base = 0x440000
stop_addr = 0x04A7 + base

dp.start(base, end=stop_addr)

emu_start(440000, 4404a7, 0)
syscall: ZwProtectVirtualMemory(
    HANDLE ProcessHandle = 0xffffffff /* NtCurrentProcess() */,
    PVOID* BaseAddress = 0x19fb1c,
    SIZE_T* RegionSize = 0x19fb20,
    ULONG NewProtect = 0x4,
    ULONG* OldProtect = 0x19fb24
)
protect 0x770f9000[0x3000] = PAGE_READWRITE
status = 0
syscall: ZwAccessCheck(
    SECURITY_DESCRIPTOR* SecurityDescriptor = 0xffffffff,
    HANDLE ClientToken = 0x19fb1c,
    ACCESS_MASK DesiredAccess = 0x19fb20,
    GENERIC_MAPPING* GenericMapping = 0x4,
    PRIVILEGE_SET* PrivilegeSet = 0x19fb24,
    ULONG* PrivilegeSetLength = 0x770f9000,
    ACCESS_MASK* GrantedAccess = 0x2378,
    NTSTATUS* AccessStatus = 0x2
)
status = 0
syscall: ZwProtectVirtualMemory(
    HANDLE ProcessHandle = 0xffffffff /* NtCurrentProcess() */,
    PVOID* BaseAddress = 0x19fb1c,
    SIZE_T* RegionSize = 0x19fb20,
    ULONG NewProtect = 0x2,
    ULONG* OldProtect = 0x19fb24
)
protect 0x770f9000[0x3000] = PAGE_READONLY
status = 0
syscall: ZwAccessCheck(
 

In [4]:
fn_get_string = 0x017DE0 + base

buff = dp.allocate(4096)

dp.call(fn_get_string, [buff])

emu_start(457de0, 5000, 0)
interrupt 3 (#BP, Breakpoint), cip = 457de1, cs = 23
handling exception...
old csp: 19fb74, new csp: 19f6bc
emu_start(77044ec0, 5000, 0)
syscall: ZwContinue(
    PVOID ContextRecord = 0x19f714,
    BOOLEAN TestAlert = 0x0
)
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
switching context, cip: 0x457df2
emu_start(457df2, 5000, 0)
interrupt 3 (#BP, Breakpoint), cip = 457df7, cs = 23
handling exception...
old csp: 19fb74, new csp: 19f6bc
emu_start(77044ec0, 5000, 0)
syscall: ZwContinue(
    PVOID ContextRecord = 0x19f714,
    BOOLEAN TestAlert = 0x0
)
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
switching context, cip: 0x457e00
emu_start(457e00, 5000, 0)
interrupt 3 (#BP, Breakpoint), cip = 457e07, cs = 23
handling exception...
old csp: 19fb74, new csp: 19f6bc
emu_start(77044ec0, 5000, 0)
syscall: ZwContinue(
    PVOID ContextRecord = 0x19f714,
    BOOLEAN TestAlert = 0x0
)
context switch, s

4838920

In [7]:
out_buff_len = dp.read_long(buff)
out_buff = dp.read(buff+4, out_buff_len)
out_buff

bytearray(b'TOnm\xf6_,\xcb\x8d\x84:\x9a\xdaW\xed\x15Bht\x91\x19\xe8\xa8)')

In [8]:
key = bytes.fromhex('3d4f0b6d845f58cbf8844e9ab35781156c68109175e8c42901f8ee2b78c4926631939c778b2a48e0d8ea0dd585')


In [10]:
def xor_crypt(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i % len(key)])
    return bytes(out)

xor_crypt(out_buff, key).decode('utf-16')


'iertutil.dll'

In [11]:
fn_decrypt = 0x00025A28  + base

dp.call(fn_decrypt, [buff])


emu_start(465a28, 5000, 0)
emulation finished, cip = 5000


4838920

In [15]:
dp.read(buff+4, out_buff_len).decode('utf-16')

'iertutil.dll'

In [24]:
str_fn_list = [0x254d4, 0x1644a, 0x147f1, 0x1607f, 0x1d6a4, 0x1be3d, 0x1babc, 0x1836d, 0x8a55, 0x8d44, 0x670f, 0x4270, 0xac35, 0xaab1, 0x8f89, 0x9d9c, 0x1d9d3, 0x14d18, 0x18aa1, 0x19765, 0x1a31c, 0x1ae60, 0x17571, 0x18f33, 0x192f4, 0x17de0, 0x15b41, 0x14d35, 0x167d4, 0x171af, 0x16e4f, 0x1cf67, 0x1c943, 0x17aa1]

# for str_fn in str_fn_list:
#     try:
#         print(decrypt(str_fn))
#     except:
#         pass

def emu_restart():
    global dp
    # Load the emulator
    dp = Dumpulator("/tmp/gudump.dmp", quiet=True)
    # Load the VEH
    veh_stop_addr = 0x04A7 + base
    dp.start(0x440000, end=veh_stop_addr)

    
def emu_decrypt(fn_addr):
    global dp
    buff = dp.allocate(4096)
    fn_get_string = fn_addr + base
    try:
        dp.call(fn_get_string, [buff])
        out_buff_len = dp.read_long(buff)
        dp.call(fn_decrypt, [buff])
        out_buff = dp.read(buff+4, out_buff_len)
        return out_buff.replace(b'\x00',b'').decode('utf-8')
    except:
        emu_restart()
        return ''
        
emu_restart()

out_strs = []
for str_fn in str_fn_list:
    out_strs.append(emu_decrypt(str_fn))
    

protect 0x770f9000[0x3000] = PAGE_READWRITE
protect 0x770f9000[0x3000] = PAGE_READONLY
initial unsupported access UC_MEM_WRITE_PROT of 1a0000[1] = 465B39, cip = 4654a4
protect 0x770f9000[0x3000] = PAGE_READWRITE
protect 0x770f9000[0x3000] = PAGE_READONLY
interrupt 3 (#BP, Breakpoint), cip = 45644f, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 45645b, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 45646f, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 45647b, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 45648a, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 4564a4, cs = 23
context switch, s

Traceback (most recent call last):
  File "/Users/herrcore/.pyenv/versions/3.9.5/lib/python3.9/site-packages/dumpulator/dumpulator.py", line 1329, in _hook_syscall
    status = syscall_impl(dp, *args)
  File "/Users/herrcore/.pyenv/versions/3.9.5/lib/python3.9/site-packages/dumpulator/ntsyscalls.py", line 2888, in ZwQueryInformationJobObject
    raise NotImplementedError()
NotImplementedError


Exception thrown during syscall implementation, stopping emulation!
forced exit memory operation 21 of 4fe2[1] = 0
protect 0x770f9000[0x3000] = PAGE_READWRITE
protect 0x770f9000[0x3000] = PAGE_READONLY
interrupt 3 (#BP, Breakpoint), cip = 458aa2, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 458aaa, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 458ac8, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 458ad1, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 458ae3, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 4fe2[1] = 0
interrupt 3 (#BP, Breakpoint), cip = 458b02, cs = 23
context switch, stopping emulation
forced exit memory operation 21 of 

In [25]:
out_strs

['',
 'user32',
 'psapi.dll',
 'Msi.dll',
 'Publisher',
 'Software\\Microsoft\\Windows\\CurrentVersion\\Run',
 'Skattekister138',
 'OverOps146.70.147.12/vSFjv98.fla',
 'SOFTWARE\\AppDataLow\\',
 'Tumorlike',
 "%PSHEX% -windowstyle minimized $Tritetartemorion=(Get-ItemProperty -Path 'HKCU:\\SOFTWARE\\AppDataLow\\').Tumorlike;%PSHEX% -windowstyle minimized ($Tritetartemorion)",
 "%PSHEX% -windowstyle minimized $Ddsensalvorligt=(Get-ItemProperty -Path 'HKCU:\\SOFTWARE\\AppDataLow\\').Tumorlike;%PSHEX% -windowstyle minimized ($Ddsensalvorligt)",
 'Environment',
 'PSHEX',
 'c:\\windows\\system32\\WindowsPowerShell\\v1.0\\powershell.exe',
 'c:\\windows\\SysWOW64\\WindowsPowerShell\\v1.0\\powershell.exe',
 '',
 '',
 'ProgramFiles=',
 '\\internet explorer\\ieinstal.exe',
 '\\internet explorer\\ielowutil.exe',
 '\\internet explorer\\ExtExport.exe',
 'windir=',
 '\\system32\\',
 '\\syswow64\\',
 'iertutil.dll',
 'wininet.dll',
 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gec

In [27]:


# Load the emulator
dp = Dumpulator("/tmp/gudump.dmp", quiet=True, trace=True)
# Load the VEH
veh_stop_addr = 0x04A7 + base
dp.start(0x440000, end=veh_stop_addr)

protect 0x770f9000[0x3000] = PAGE_READWRITE
protect 0x770f9000[0x3000] = PAGE_READONLY


In [39]:
trace_file = open('/tmp/gudump.dmp.trace','rb').read()

trace_addr = []

for trace_line in trace_file.split(b'\n'):
    try:
        t_addr = int(trace_line.decode('utf-8').split('|')[0], 16)
        if  0x440000 <= t_addr < 0x440000 + 0x2a000:
            trace_addr.append(t_addr - base)
    except:
        t_addr = int(trace_line.decode('utf-8').split(' ')[0], 16)
        if  0x440000 <= t_addr < 0x440000 + 0x2a000:
            trace_addr.append(t_addr - base)

In [42]:
trace_addr = list(set(trace_addr))
print(trace_addr)

[0, 153600, 149504, 153603, 149510, 149512, 153609, 153612, 149518, 149519, 153618, 149525, 149527, 153624, 153627, 149533, 153629, 149538, 153635, 153641, 153643, 153649, 153655, 165946, 153660, 165949, 153662, 153668, 153670, 153676, 153678, 153684, 149604, 149606, 149611, 149613, 149618, 149623, 149625, 149631, 149637, 149639, 149640, 166023, 149642, 166028, 149647, 166034, 149652, 168085, 149658, 149663, 149668, 153766, 149670, 153768, 149673, 149676, 153774, 149678, 153780, 149684, 153782, 149686, 153785, 149689, 153787, 149691, 149694, 149699, 149700, 149703, 149705, 149711, 149717, 149722, 149727, 149729, 149734, 149739, 149741, 149747, 149749, 149753, 149755, 149757, 149759, 257, 149764, 149769, 166156, 149775, 149776, 166159, 149778, 149779, 166161, 149781, 166164, 149787, 149789, 149795, 149797, 149798, 149804, 149806, 149811, 149813, 149820, 149823, 149830, 149837, 149839, 149842, 149848, 149853, 149858, 149864, 149870, 166262, 375, 166263, 166265, 380, 166271, 149952, 450, 

#### IDA Label Trace

```python
import idc 
trace_addr = [0, 153600, 149504, 153603, 149510, 149512, 153609, 153612, 149518, 149519, 153618, 149525, 149527, 153624, 153627, 149533, 153629, 149538, 153635, 153641, 153643, 153649, 153655, 165946, 153660, 165949, 153662, 153668, 153670, 153676, 153678, 153684, 149604, 149606, 149611, 149613, 149618, 149623, 149625, 149631, 149637, 149639, 149640, 166023, 149642, 166028, 149647, 166034, 149652, 168085, 149658, 149663, 149668, 153766, 149670, 153768, 149673, 149676, 153774, 149678, 153780, 149684, 153782, 149686, 153785, 149689, 153787, 149691, 149694, 149699, 149700, 149703, 149705, 149711, 149717, 149722, 149727, 149729, 149734, 149739, 149741, 149747, 149749, 149753, 149755, 149757, 149759, 257, 149764, 149769, 166156, 149775, 149776, 166159, 149778, 149779, 166161, 149781, 166164, 149787, 149789, 149795, 149797, 149798, 149804, 149806, 149811, 149813, 149820, 149823, 149830, 149837, 149839, 149842, 149848, 149853, 149858, 149864, 149870, 166262, 375, 166263, 166265, 380, 166271, 149952, 450, 149954, 452, 149960, 149962, 149963, 149969, 149972, 149974, 149977, 149980, 149985, 149990, 149996, 149998, 150000, 150004, 166389, 166391, 150009, 150012, 150014, 150016, 150017, 150019, 150021, 150026, 150027, 150029, 150032, 534, 536, 538, 150043, 150058, 150061, 150064, 150067, 150069, 150072, 150074, 150077, 150079, 150081, 150084, 150086, 150089, 150092, 150094, 150097, 150099, 150102, 150105, 150108, 150111, 150113, 150116, 150118, 150120, 166505, 150123, 166508, 150125, 150131, 628, 150133, 630, 150135, 632, 150136, 634, 150138, 640, 150144, 150146, 646, 648, 150152, 650, 150154, 652, 653, 654, 150155, 656, 150161, 659, 150164, 661, 663, 665, 150170, 667, 150172, 670, 673, 675, 166570, 166573, 166574, 166580, 150254, 150259, 150260, 760, 150265, 762, 764, 765, 150271, 768, 770, 150274, 773, 150279, 776, 150281, 778, 150282, 780, 166666, 782, 150283, 166672, 166669, 150286, 150292, 790, 150294, 150296, 793, 795, 150299, 150303, 801, 804, 150309, 150311, 809, 150314, 150315, 814, 815, 150321, 821, 150326, 150328, 826, 150331, 832, 150337, 835, 841, 843, 150347, 150349, 150352, 849, 150356, 855, 150359, 150361, 858, 168795, 860, 150364, 168799, 150368, 168801, 150370, 866, 168802, 150373, 868, 168804, 168805, 871, 168807, 873, 168809, 168811, 150375, 150378, 879, 168816, 881, 150381, 150383, 884, 150386, 168822, 887, 150389, 168825, 890, 150391, 892, 168831, 168833, 897, 900, 168839, 903, 168841, 168843, 908, 168845, 168847, 913, 168850, 168852, 168854, 918, 168856, 924, 929, 150435, 932, 935, 150439, 937, 150444, 942, 150447, 945, 150452, 950, 150457, 955, 150459, 150461, 960, 962, 150466, 964, 150468, 967, 970, 150474, 166858, 973, 150477, 975, 150479, 166860, 166862, 166865, 980, 150485, 166867, 150487, 166869, 150489, 985, 988, 150495, 991, 996, 150501, 999, 150507, 1004, 150513, 150518, 150521, 150522, 150527, 150533, 150539, 150540, 150546, 150549, 150555, 150561, 150563, 150569, 150575, 1074, 1076, 166967, 166969, 1082, 1084, 166974, 166976, 1091, 1092, 1093, 166979, 1095, 166982, 166983, 166988, 1101, 1103, 166991, 166993, 1106, 1107, 166999, 1113, 167001, 1115, 1117, 1119, 167007, 1121, 167009, 1127, 167015, 1130, 167018, 167019, 167022, 1135, 167025, 1138, 167027, 1140, 167029, 167031, 167033, 1148, 167038, 167040, 167043, 167046, 150665, 150667, 167052, 167055, 150673, 150675, 167061, 150678, 150684, 150690, 150692, 150698, 167082, 150700, 167083, 167085, 167088, 150705, 150708, 167095, 167097, 150714, 150716, 150718, 167102, 150720, 150723, 150725, 167109, 167111, 150728, 150730, 167118, 150735, 167120, 150737, 152788, 150741, 152790, 150743, 150744, 167127, 167130, 152795, 152796, 167132, 150750, 152802, 150755, 167138, 167140, 150758, 167143, 152808, 150760, 167145, 152811, 152813, 150766, 167152, 152819, 150772, 152821, 167156, 167158, 167160, 150778, 152827, 152829, 150781, 152831, 150782, 167165, 152834, 150787, 152836, 167167, 152838, 167172, 167175, 150793, 152843, 167180, 152845, 167182, 150799, 150800, 167184, 152851, 150806, 167191, 152857, 150809, 150811, 152860, 167198, 150817, 152866, 152868, 167204, 152870, 150823, 167207, 152873, 152875, 152880, 152882, 152888, 152894, 152900, 166762, 167281, 167284, 167293, 166766, 167302, 152970, 152973, 167311, 152976, 152979, 152981, 152983, 167320, 152985, 167322, 152988, 167325, 152990, 166772, 152992, 152995, 167332, 167338, 167340, 153017, 153019, 153023, 153025, 153031, 153034, 153039, 153043, 153049, 153054, 153060, 153062, 153068, 153071, 153074, 153080, 153082, 153084, 153086, 153091, 153097, 153099, 153105, 153107, 153113, 153116, 153119, 153125, 153130, 153136, 153142, 153148, 153150, 153154, 153156, 153158, 153164, 153170, 153173, 153179, 153183, 153185, 153186, 153191, 153197, 153199, 153204, 153210, 153213, 153217, 153219, 153221, 153227, 153233, 153235, 153240, 153242, 153248, 153250, 153256, 153262, 153265, 153268, 153273, 153279, 153285, 153291, 153385, 153387, 153390, 153396, 153406, 153410, 153412, 153417, 153423, 153429, 153435, 153436, 153459, 153460, 149373, 149377, 149379, 149381, 149384, 153483, 149389, 149391, 153488, 153489, 153491, 153494, 153496, 153499, 153502, 153503, 153508, 153510, 153516, 153517, 153518, 153520, 153524, 153526, 153532, 153537, 153540, 153546, 153550, 165839, 153556, 165844, 165845, 165848, 149465, 153562, 149467, 153565, 153567, 149473, 153570, 149475, 149477, 149478, 153576, 153578, 153580, 149484, 149486, 153585, 149492, 153589, 149494, 149495, 149496, 153595, 149502]
for addr in trace_addr:
    idc.create_insn(addr)
    set_color(addr, CIC_ITEM, color_value)
```